In [19]:
import pandas as pd
import seaborn as sns
import numpy as np
import lightgbm as lgb
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

def rmsle(y_true, y_pred):
    """Root Mean Squared Logarithmic Error"""
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2))


# input, target, pca 주성분 개수를 입력 받아 dataframe의 주성분 분석을 통해, n개의 주성분으로 구성된 새로운 데이터 세트를 만들어 내는 함수
def convert_pca_n_df (df_x, df_y, n):
    pca = PCA(n_components=df_x.shape[1])  # 모든 성분을 유지
    X_pca = pca.fit_transform(df_x)
    
    # 성분별 설명 분산 비율
    # explained_variance = pca.explained_variance_ratio_
    
    # PCA 성분들과 sales 사이의 상관관계 확인
    pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(df_x.shape[1])])
    pca_df['sales'] = df_y.values
    
    # 성분들과 sales 간의 상관계수 계산
    correlations = pca_df.corr()['sales'].drop('sales').abs()
    
    # sales에 가장 큰 영향을 주는 성분들을 선택
    important_components = correlations.sort_values(ascending=False)
    
    # 상위 몇 개의 성분을 기준으로 features 축소
    n_important = n  # 상위 5개의 중요한 성분을 선택하는 경우
    selected_components = important_components.index[:n_important]
    df_result = pca_df[selected_components]
    
    return df_result


In [22]:
# 전처리(Label Encoding)되 데이터 로드
#X_train = pd.read_csv('TestData/train_data_label_encoded.csv')
X_train = pd.read_csv('TestData/X_test_pca_42.csv')
y_train = pd.read_csv('TestData/train_target.csv').iloc[:, 0]
# X_val = pd.read_csv('TestData/val_data_label_encoded.csv')
X_val = pd.read_csv('TestData/X_val_pca_42.csv')
y_val = pd.read_csv('TestData/val_target.csv').iloc[:, 0]

# 학습에 사용하지 않을 컬럼 삭제
# X_train.drop(['id', 'date', 'date_type', 'day'], axis=1, inplace=True)
# X_val.drop(['id', 'date', 'date_type', 'day'], axis=1, inplace=True)

In [14]:
# PCA 주성분 분석 및 sales와의 상관관계 분석을 통해 feature 축소

# n=10 개의 주성분 추출
X_train_pca = convert_pca_n_df (X_train, y_train, 10)
X_val_pca = convert_pca_n_df (X_val, y_val, 10)

In [23]:
# LightGBM 모델 생성
model = lgb.LGBMRegressor()

In [24]:
# GVSearchCV로 LGBMRegressor 모델을 최적화하기 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 63, 127],
    'max_depth': [-1, 10, 20]
}


In [25]:
# 모델을 평가하기 위한 rmsle 스코어 함수 생성
rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

# GridSearchCV 객체 생성
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=rmsle_scorer, verbose=1, n_jobs=-1)

# GridSearchCV 학습
grid_search.fit(X_train_pca, y_train)

# 최적 하이퍼파라미터 및 성능 출력
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# 최적 모델로 테스트 데이터 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val_pca)
y_pred = np.clip(y_pred, a_min=0, a_max=None)

# Calculate RMSLE
score = rmsle(y_val, y_pred)
print("RMSLE:", score)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017414 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 2945646, number of used features: 10
[LightGBM] [Info] Start training from score 355.629570
Best Parameters: {'learning_rate': 0.2, 'max_depth': -1, 'n_estimators': 100, 'num_leaves': 127}
Best Score: -2.2658357253063492
RMSLE: 2.05987620647148
